# Customer Churn problem With Artificial Neural Network

### Importing the libraries

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
tf.__version__

'2.18.0'

## Part 1 - Data Preprocessing

### Importing the dataset

In [3]:
df = pd.read_csv('Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
# Split the data
X = df.iloc[:, 3:-1].values # There is no use with first 3 columns so we avoid using first 3 columns in X data
y = df.iloc[:, -1].values # This is our target value

In [5]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [6]:
print(y)

[1 0 1 ... 1 1 0]


### Encoding categorical data

Label Encoding the "Gender" column

In [7]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

In [8]:
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


One Hot Encoding the "Geography" column

In [9]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers = [('encoder', OneHotEncoder(), [1])], remainder = 'passthrough')
X = np.array(ct.fit_transform(X))

### Splitting the dataset into the Training set and Test set

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size = 0.2,
                                                    random_state = 40)

In [11]:
X_train, X_test, y_train, y_test

(array([[0.0, 1.0, 0.0, ..., 0, 1, 59595.66],
        [1.0, 0.0, 0.0, ..., 1, 0, 20629.4],
        [0.0, 1.0, 0.0, ..., 0, 1, 52314.71],
        ...,
        [0.0, 0.0, 1.0, ..., 1, 0, 20016.59],
        [0.0, 0.0, 1.0, ..., 1, 0, 43134.58],
        [0.0, 0.0, 1.0, ..., 0, 1, 13998.88]], dtype=object),
 array([[0.0, 0.0, 1.0, ..., 0, 0, 150401.53],
        [0.0, 1.0, 0.0, ..., 0, 0, 31730.32],
        [1.0, 0.0, 0.0, ..., 0, 1, 197297.77],
        ...,
        [1.0, 0.0, 0.0, ..., 1, 0, 87347.82],
        [0.0, 0.0, 1.0, ..., 0, 0, 3946.29],
        [1.0, 0.0, 0.0, ..., 1, 0, 94263.44]], dtype=object),
 array([0, 1, 0, ..., 0, 0, 0]),
 array([1, 0, 1, ..., 0, 0, 0]))

### Feature Scaling

In [12]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [13]:
X_train, X_test

(array([[-1.00601811,  1.73089688, -0.57234647, ..., -1.55337352,
          0.97067965, -0.70848497],
        [ 0.99401789, -0.57773517, -0.57234647, ...,  0.64376017,
         -1.030206  , -1.38762036],
        [-1.00601811,  1.73089688, -0.57234647, ..., -1.55337352,
          0.97067965, -0.83538323],
        ...,
        [-1.00601811, -0.57773517,  1.74719344, ...,  0.64376017,
         -1.030206  , -1.3983009 ],
        [-1.00601811, -0.57773517,  1.74719344, ...,  0.64376017,
         -1.030206  , -0.99538194],
        [-1.00601811, -0.57773517,  1.74719344, ..., -1.55337352,
          0.97067965, -1.5031824 ]]),
 array([[-1.00601811, -0.57773517,  1.74719344, ..., -1.55337352,
         -1.030206  ,  0.87415292],
        [-1.00601811,  1.73089688, -0.57234647, ..., -1.55337352,
         -1.030206  , -1.19414457],
        [ 0.99401789, -0.57773517, -0.57234647, ..., -1.55337352,
          0.97067965,  1.69149839],
        ...,
        [ 0.99401789, -0.57773517, -0.57234647, ...,  

In [14]:
y_train

array([0, 1, 0, ..., 0, 0, 0])

## Part 2 - Building the ANN

### Initializing the ANN

In [15]:
ann = tf.keras.models.Sequential()

### Adding the input layer and the first hidden layer

In [38]:
ann.add(tf.keras.layers.Dense(units = 6, activation = 'relu'))

### Adding the second hidden layer

In [39]:
ann.add(tf.keras.layers.Dense(units = 6, activation = 'relu'))

### Adding the output layer

In [40]:
ann.add(tf.keras.layers.Dense(1, activation = 'sigmoid'))

## Part 3 - Training the ANN

### Compiling the ANN

In [41]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the ANN on the Training set

In [42]:
history = ann.fit(X_train, y_train, batch_size = 32, epochs = 100, verbose = 1)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8197 - loss: 0.6364
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8616 - loss: 0.3897
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8578 - loss: 0.3536
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8630 - loss: 0.3435
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.8717 - loss: 0.3264
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8613 - loss: 0.3412
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8585 - loss: 0.3466
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.8620 - loss: 0.3432
Epoch 9/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.8693 - loss: 0.3303
Epoch 10/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8604 - loss: 0.3414
Epoch 11/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8633 - loss: 0.3401
Epoch 12/100
250/250 ━━━━━━━━━━━━━━━━━━━━

In [52]:
loss, accuracy = history.history['loss'], history.history['accuracy']
[loss[-1:], accuracy[-1:]]

[[0.33445948362350464], [0.8611249923706055]]

## Part 4 - Making the predictions and evaluating the model

### Predicting the result of a single observation

**Homework**

Use our ANN model to predict if the customer with the following informations will leave the bank:

Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: \$ 60000

Number of Products: 2

Does this customer have a credit card? Yes

Is this customer an Active Member: Yes

Estimated Salary: \$ 50000

So, should we say goodbye to that customer?

**Solution**

In [46]:
print(ann.predict(sc.transform([[1.0, 0.0, 0.0, 600, 1, 40, 3, 6000, 2, 1, 1, 50000]])) > 0.5)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
[[False]]


Therefore, our ANN model predicts that this customer stays in the bank!

**Important note 1:** Notice that the values of the features were all input in a double pair of square brackets. That's because the "predict" method always expects a 2D array as the format of its inputs. And putting our values into a double pair of square brackets makes the input exactly a 2D array.

**Important note 2:** Notice also that the "France" country was not input as a string in the last column but as "1, 0, 0" in the first three columns. That's because of course the predict method expects the one-hot-encoded values of the state, and as we see in the first row of the matrix of features X, "France" was encoded as "1, 0, 0". And be careful to include these values in the first three columns, because the dummy variables are always created in the first columns.

### Predicting the Test set results

In [45]:
y_preds = ann.predict(X_test)
y_preds = (y_preds > 0.5)
print(np.concatenate((y_preds.reshape(len(y_preds), 1), y_test.reshape(len(y_test), 1)), 1))

63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step
[[0 1]
 [0 0]
 [0 1]
 ...
 [0 0]
 [0 0]
 [0 0]]


### Making the Confusion Matrix

In [54]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_preds)
print(cm)
accuracy_score(y_test, y_preds)

[[1557   59]
 [ 197  187]]


0.872